### Connect to workspace

In [5]:
import azureml.core
from azureml.core import Workspace
print(azureml.core.VERSION)

1.0.39


In [6]:
ws = Workspace.from_config()

In [7]:
print(ws.name, ws.location, ws.resource_group, ws.location, sep = '\t')

prodmod	westus2	prodmod_rg	westus2


### Upload data

In [14]:
ds = ws.get_default_datastore()

In [15]:
ds.upload(src_dir='../data', target_path='bf_data', overwrite=False, show_progress=True)

Uploading ../data\test.csv
Uploading ../data\train.csv
Uploading ../data\training_data.csv
Uploaded ../data\test.csv, 1 files out of an estimated total of 3
Uploaded ../data\train.csv, 2 files out of an estimated total of 3
Uploaded ../data\training_data.csv, 3 files out of an estimated total of 3


$AZUREML_DATAREFERENCE_2ab1f20d64b84545bc5a0860673e4aea

### Create an experiment

In [9]:
experiment_name = 'keras-black-friday'

from azureml.core import Experiment
exp = Experiment(workspace=ws, name=experiment_name)

### Train on compute nodes

In [11]:
compute_target = ws.compute_targets['prodmod-compute1']

In [12]:
from azureml.train.dnn import TensorFlow

In [13]:
script_folder = '.'

In [31]:
script_params = {
    '--data-folder': ds.path('bf_data').as_mount(),
    '--batch-size': 512,
    '--first-layer-neurons': 512,
    '--second-layer-neurons': 256,
    '--learning-rate': 0.01
}

In [32]:
estimator = TensorFlow(source_directory=script_folder,
                script_params=script_params,
                compute_target=compute_target,
                pip_packages=['keras', 'pandas', 'numpy', 'sklearn'],
                entry_script='train.py',
                use_gpu=False)

framework_version is not specified, defaulting to version 1.13.


In [33]:
run = exp.submit(estimator)

In [34]:
from azureml.widgets import RunDetails
RunDetails(run).show()

_UserRunWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO', '…

### Hyper parameter tuning

In [35]:
model = run.register_model(model_name='keras_bf', model_path='outputs/model')

### Make a prediction

In [44]:
os.makedirs('./model', exist_ok=True)

for f in run.get_file_names():
    if f.startswith('outputs/model'):
        output_file_path = os.path.join('./model', f.split('/')[-1])
        print('Downloading from {} to {} ...'.format(f, output_file_path))
        run.download_file(name=f, output_file_path=output_file_path)

In [58]:
import pickle
import numpy as np
from tensorflow.keras.models import model_from_json

In [42]:
with open('../data/test_val.pkl', 'rb') as f:
    test_val = pickle.load(f)

In [64]:
test_val

{'User_ID': '1000001',
 'Product_ID': 'P00248942',
 'Gender': 'F',
 'Age': '0-17',
 'Occupation': '10',
 'City_Category': 'A',
 'Stay_In_Current_City_Years': '2',
 'Marital_Status': '0',
 'Product_Category_1': '1',
 'Product_Category_2': '6.0',
 'Product_Category_3': '14.0',
 'Purchase': 15200}

In [45]:
with open('./model/index_map.pkl', 'rb') as f:
    index_map = pickle.load(f)

In [47]:
with open('./model/outcome_var.pkl', 'rb') as f:
    target_col = pickle.load(f)

In [48]:
test_data = [[index_map[key][value]] for key, value in test_val.items() if key!=target_col]

In [49]:
test_data

[[0], [1], [0], [0], [0], [0], [0], [0], [1], [1], [1]]

In [51]:
with open('./model/model.json', 'r') as f:
    loaded_model_json = f.read()

In [55]:
loaded_model = model_from_json(loaded_model_json)

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


In [56]:
loaded_model.load_weights("./model/model.h5")

In [62]:
pred = int(np.squeeze(loaded_model.predict(test_data)))

In [63]:
{'predicted': pred, 'actual': test_val['Purchase']}

{'predicted': 17175, 'actual': 15200}

### Deploy model to service

In [174]:
from azureml.core.runconfig import CondaDependencies

cd = CondaDependencies.create()
cd.add_conda_package('tensorflow')
cd.add_conda_package('keras')
cd.save_to_file(base_directory='./', conda_file_path='myenv.yml')

print(cd.serialize_to_string())

# Conda environment specification. The dependencies defined in this file will
# be automatically provisioned for runs with userManagedDependencies=False.

# Details about the Conda environment file format:
# https://conda.io/docs/user-guide/tasks/manage-environments.html#create-env-file-manually

name: project_environment
dependencies:
  # The python interpreter version.
  # Currently Azure ML only supports 3.5.2 and later.
- python=3.6.2

- pip:
  - azureml-defaults==1.0.39.*
- tensorflow
- keras



In [175]:
from azureml.core.webservice import AciWebservice

aciconfig = AciWebservice.deploy_configuration(cpu_cores=1, 
                                               auth_enabled=True,
                                               memory_gb=1, 
                                               tags={'name':'black_friday', 'framework': 'Keras'},
                                               description='Keras model for black friday data')

In [176]:
from azureml.core.image import ContainerImage

imgconfig = ContainerImage.image_configuration(execution_script="score.py", 
                                               runtime="python", 
                                               conda_file="myenv.yml")

In [178]:
from azureml.core.webservice import Webservice

In [179]:
%%time

service = Webservice.deploy_from_model(workspace=ws,
                                       name='keras-bf-svc',
                                       deployment_config=aciconfig,
                                       models=[model],
                                       image_config=imgconfig)

service.wait_for_deployment(show_output=True)

Creating image

Image creation operation finished for image keras-bf-svc:7, operation "Succeeded"
Creating service
Running........................
SucceededACI service creation operation finished, operation "Succeeded"
Wall time: 8min 3s


In [180]:
%%time
image = ContainerImage.create(name = "bfimage",
                              models = [model],
                              image_config = imgconfig,
                              workspace = ws)

image.wait_for_creation(show_output = True)

Creating image
Running........................................................................
Succeeded
Image creation operation finished for image bfimage:4, operation "Succeeded"
Wall time: 6min 17s


### Test service

In [126]:
with open('../data/test_val.pkl', 'rb') as f:
    test_val = pickle.load(f)

In [127]:
with open('./model/outcome_var.pkl', 'rb') as f:
    target_col = pickle.load(f)

In [132]:
actual = test_val[target_col]

In [159]:
test_data = {k:v for k,v in test_val.items() if not k == target_col}

In [171]:
test_data_json = bytes(json.dumps(test_data), encoding='utf8')

In [181]:
test_data_json

b'{"User_ID": "1000001", "Product_ID": "P00248942", "Gender": "F", "Age": "0-17", "Occupation": "10", "City_Category": "A", "Stay_In_Current_City_Years": "2", "Marital_Status": "0", "Product_Category_1": "1", "Product_Category_2": "6.0", "Product_Category_3": "14.0"}'

In [182]:
service.run(input_data=test_data_json)

17175

In [183]:
actual

15200

In [184]:
print(service.scoring_uri)

http://5298bb33-0d7b-4376-ab1d-c04e79f718df.westus.azurecontainer.io/score


#### Using Post

In [187]:
key1, Key2 = service.get_keys()

In [188]:
key1

'O13psE1xgbdmsZpVjyzE8BZs69dh6uBw'

In [189]:
import requests

headers = {'Content-Type':'application/json', 'Authorization': 'Bearer ' + key1}

resp = requests.post(service.scoring_uri, test_data_json, headers=headers)

print("POST to url", service.scoring_uri)
print("prediction:", resp.text)

POST to url http://5298bb33-0d7b-4376-ab1d-c04e79f718df.westus.azurecontainer.io/score
prediction: 17175


### Cleanup

In [177]:
service.delete()